# Basic Web Data Scraping and HTML Parsing
### The purpose of this notebook is to review how to do some basic data scraping from a website. I will use requests to recieve the data from the URL and  Beautiful Soup to parse the HTML. Pandas will be used to simplify the exploratory data analysis. In the end Bokeh is utilized for visualization

In [1]:
from bs4 import BeautifulSoup
import requests, pandas


#### First step is to locate the desired data, for the following data I am selecting the html output from a link that would download the data as an Excel file. requests.get will open the url provided and store the servers response into the variable 'web', web_data is simply the text attribute of server response.

In [2]:
web =  requests.get('https://services.aamc.org/tsfreports/report_excel.cfm?select_control=PRI&year_of_study=2016')
web_data = web.text

#### Next step is to apply Beautiful Soup to have the resulting html in a format that facilitates easy navigation.

In [3]:
soup = BeautifulSoup(web_data,'lxml')

#### The 'soup' object has methods that allows the user to easily navigate up and down the HTML Tree. The method findAll is searching the tree for elements that posses the tag 'tr'. The method returns a list of elements in the tree with the tag 'tr'. We can see below the first element is a list of column names.

In [4]:
soup.findAll('tr')[0]

<tr><td><strong>SCHOOL NAME</strong></td><td><strong>PUBLIC/PRIVATE</strong></td><td><strong>RESIDENT TUITION</strong></td>
<td><strong>NONRESIDENT TUITION</strong></td><td><strong>RESIDENT FEE</strong></td>
<td><strong>NONRESIDENT FEE</strong></td>
<td><strong>RESIDENT HEALTH INSURANCE</strong></td>
<td><strong>NONRESIDENT HEALTH INSURANCE</strong></td>
<td><strong>REQUIRED</strong></td>
<td><strong>WAIVED</strong></td><td><strong>RESIDENT TUITION &amp; FEES &amp; HLTH. INS.</strong></td>
<td><strong>NONRESIDENT TUITION &amp; FEES &amp; HLTH. INS.</strong></td></tr>

#### Examining the next element in the list we see that it is the first record of the table. (Download the excel file using the URL to compare the data)

In [5]:
soup.findAll('tr')[1]

<tr>
<td>Albany</td>
<td>Private</td>
<td align="right">55360</td>
<td align="right">55360</td>
<td align="right">1840</td>
<td align="right">1840</td>
<td align="right">3168</td>
<td align="right">3168</td>
<td>YES</td><td>YES</td>
<td align="right">60368</td>
<td align="right">60368</td>
</tr>

#### Now that we have figured out how to navigate to a record in the HTML tree structure we now would like to extract the data for a record. We can use the methods provided by BeautifulSoup to navigate this tree structure to the data component we are interested. In this case, I am interested in extracting the numbers in the last column. Since each data element is encapsulated in a 'td' tag. I can apply the findAll method again on the resulting child element. then select the last item in the list.

In [6]:
foo = soup.findAll('tr')[1].findAll('td')[11]
foo

<td align="right">60368</td>

#### Now that we have isolated the exact HTML element, we can extract the data by referencing the contents attribute on the HTML element, which return the data as a string in a list.

In [7]:
bar = foo.contents
bar[0]

'60368'

### As human, we are too lazy to write out these commands to extract each element individually so we will try to write a few lines of code to do this for us.
1. Isolate Data records, by using the 'tr' tag, discard first row since it is the column headers which we are already know.
2. Initialize empty lists that will store data from each record.
3. Using a for-loop iterate over each record to get the school's name, the cost of tuition for a resident (2nd to last column) and the non-resident tuition (the last column).
4. Organize the resulting lists into a dictionary and then create a pandas.DataFrame to facilitate further analysis.

In [8]:
#Step 1.
uni_set = soup.findAll('tr')[1:]

#Step 2.
school = []
resident = []
nonresident = []

#Step 3.
for uni in uni_set:
    uni_data = uni.findAll('td')
    school.append(uni_data[0].contents[0])
    resident.append(uni_data[len(uni_data)-2].contents[0])
    nonresident.append(uni_data[len(uni_data)-1].contents[0])

#Step 4.
dict_frame={'school':school,'resident':resident,'non-resident':nonresident}
data_frame = pandas.DataFrame(dict_frame)
data_frame.head(10)

,non-resident,resident,school
0,60368,60368,Albany
1,35050,21950,Baylor
2,59171,59171,Boston
3,57779,57779,Brown-Alpert
4,56500,56500,California Northstate
5,48480,40475,Caribe
6,59303,59303,Case Western Reserve
7,55190,55190,Chicago Med Franklin
8,54448,54448,Chicago-Pritzker
9,61485,61485,Columbia


#### Pandas.DataFrame helps with some of the exploratory analysis with useful data manipulation methods that you can call on the DataFrame object. For example in order to answer: What are the top 5 most expensive University Tuitions for students classified as residents?

In [9]:
data_frame.sort_values(by='resident',ascending=False).head(5)

,non-resident,resident,school
49,63321,63321,Tufts
13,62502,62502,Dartmouth-Geisel
50,61616,61616,Tulane
9,61485,61485,Columbia
0,60368,60368,Albany


### The fun doesn't end!!!
#### Now that we have figured out how to collect and manipulate data from this website, it is time to get more data! The example above demonstrated how to get tuition data for Private Universities in 2016. Using the basic principles of deduction lets alter the URL and get both the private and public data for all the years available (1997-2016). Then we could compare tuition changes at universities over time!

In [10]:
years = range(1997,2017,1)
uni_type =['PRI','PUB']
iter_set = [(x,y) for x in years for y in uni_type]

data_set = []

for year_typ in iter_set:
    year = year_typ[0]
    uni_typ = year_typ[1]
    web =  requests.get('https://services.aamc.org/tsfreports/report_excel.cfm?select_control=%s&year_of_study=%s'%(uni_typ,year))
    web_data = web.text
    soup = BeautifulSoup(web_data,'lxml')

    uni_set = soup.findAll('tr')[1:]

    school = []
    resident = []
    nonresident = []
    for uni in uni_set:
        uni_data = uni.findAll('td')
        school.append(uni_data[0].contents[0])
        resident.append(uni_data[len(uni_data)-2].contents[0])
        nonresident.append(uni_data[len(uni_data)-1].contents[0])
    #import pandas
    tmp_df = pandas.DataFrame({'school':school,'resident':resident,'non_resident':nonresident})
    tmp_df['year'] = year
    tmp_df['uni_type'] = uni_typ
    data_set.append(tmp_df)

#### We aggregate all the individual DataFrames into one named dat then  we view the first 5 recods the new DataFrame.

In [11]:
dat = pandas.concat(data_set)
dat.head()

,non_resident,resident,school,year,uni_type
0,27593,26258,Albany,1997,PRI
1,21427,8152,Baylor,1997,PRI
2,31925,31925,Boston,1997,PRI
3,26045,26045,Brown-Alpert,1997,PRI
4,24860,17860,Caribe,1997,PRI


#### When the tuition data was read from html object in BeautifulSoup the data type was a string but to do further analysis we need to convert the tuition columns in the DataFrame to integers.

In [12]:
dat.resident = pandas.to_numeric(dat.resident, errors='coerce')
dat.non_resident = pandas.to_numeric(dat.non_resident, errors='coerce')

### Visualization Time!
#### I prefer to use Bokeh library because it produces html objects and interactive charts.

In [13]:
%matplotlib inline
from bokeh.io import output_notebook, show, hplot
from bokeh.charts import Line
output_notebook()

BokehJS successfully loaded.

#### Below we can create plots for the public universities and private universities resident tuition costs over time.

In [14]:
pub_plot = Line(dat[dat.uni_type=='PUB'], x='year', y='resident',
               color='school',
               legend='top_left', title='Public School Resident Tuition')

pri_plot = Line(dat[dat.uni_type=='PRI'], x='year', y='resident',
               color='school',
               legend='top_left', title='Private School Resident Tuition')

plots = hplot(pub_plot,pri_plot)

show(plots)

#### Those charts are too messy for us to be able to pick out individual universites to observe over time. Instead lets plot the top 5 and bottom 5 universities by resident tuition so we can see if there are any trends over between the cohorts. First we need to do a little data manipulation to pick out the universities by being the most expensive or less expensive and if the school is private or public.

In [15]:
pub_unis = dat.loc[(dat.uni_type=='PUB')&(dat.year==2016)].sort_values(by='resident',ascending=False)
top_pub = pub_unis.iloc[0:5,:]
top_pub = list(top_pub.school)

bottom_pub = pub_unis.iloc[-5:,:]
bottom_pub = list(bottom_pub.school)

pri_unis = dat.loc[(dat.uni_type=='PRI')&(dat.year==2016)].sort_values(by='resident',ascending=False)
top_pri = pri_unis.iloc[0:5,:]
top_pri = list(top_pri.school)

bottom_pri = pri_unis.iloc[-5:,:]
bottom_pri = list(bottom_pri.school)

In [16]:
pub_top_plot = Line(dat[dat.school.isin(top_pub)], x='year', y='resident',
               color='school',
               legend='top_left', title='Top 5 Most Expensive Public Schools for residents', ylabel='Tuition')

pub_bottom_plot = Line(dat[dat.school.isin(bottom_pub)], x='year', y='resident',
               color='school',
               legend='top_left', title='Top 5 Least Expensive Public Schools for residents', ylabel='Tuition')

pub_plots = hplot(pub_top_plot, pub_bottom_plot)
show(pub_plots)

In [17]:
pri_top_plot = Line(dat[dat.school.isin(top_pri)], x='year', y='resident',
               color='school',
               legend='top_left', title='Top 5 Most Expensive Private Schools',ylabel='Tuition')

pri_bottom_plot = Line(dat[dat.school.isin(bottom_pri)], x='year', y='resident',
               color='school',
               legend='top_left', title='Top 5 Least Expensive Private Schools',ylabel='Tuition')

pri_plots = hplot(pri_top_plot, pri_bottom_plot)
show(pri_plots)